In [ ]:
!pip install statsmodels
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 13.3 MB/s eta 0:00:00


In [ ]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
orders["day_hour"] = orders["order_dow"].astype(str) + "-" + orders["order_hour_of_day"].astype(str)
orders["user_day"] = orders["user_id"].astype(str) + "-" + orders["order_dow"].astype(str)
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,day_hour,user_day
0,2539329,1,prior,1,2,8,NaN,2-8,1-2
1,2398795,1,prior,2,3,7,15.0,3-7,1-3
2,473747,1,prior,3,3,12,21.0,3-12,1-3
3,2254736,1,prior,4,4,7,29.0,4-7,1-4
4,431534,1,prior,5,4,15,28.0,4-15,1-4


In [ ]:
orders = orders[orders["eval_set"]=="prior"]
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,day_hour,user_day
0,2539329,1,prior,1,2,8,NaN,2-8,1-2
1,2398795,1,prior,2,3,7,15.0,3-7,1-3
2,473747,1,prior,3,3,12,21.0,3-12,1-3
3,2254736,1,prior,4,4,7,29.0,4-7,1-4
4,431534,1,prior,5,4,15,28.0,4-15,1-4


In [ ]:
orders.shape

(3214874, 9)

In [ ]:
orders.isnull().sum()

,0
order_id,0
user_id,0
eval_set,0
order_number,0
order_dow,0
order_hour_of_day,0
days_since_prior_order,206209
day_hour,0
user_day,0


In [ ]:
order_products = pd.read_csv('order_products__prior.csv')
order_products

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [ ]:
order_products.isnull().sum()


,0
order_id,0
product_id,0
add_to_cart_order,0
reordered,0


In [ ]:
df_merged = pd.merge(orders,order_products, how="inner", on="order_id")[["order_id","user_id","product_id","day_hour","user_day"]]
df_merged.head()

,order_id,user_id,product_id,day_hour,user_day
0,2539329,1,196,2-8,1-2
1,2539329,1,14084,2-8,1-2
2,2539329,1,12427,2-8,1-2
3,2539329,1,26088,2-8,1-2
4,2539329,1,26405,2-8,1-2


In [ ]:
df_merged["product_id"].value_counts()
df_merged["product_id"].value_counts().mean()
low_conf, up_conf = sms.DescrStatsW(df_merged["product_id"].value_counts()).tconfint_mean()
print(f"Lower Confidence Interval: {low_conf:.0f}")
print(f"Upper Confidence Interval: {up_conf:.0f}")

Lower Confidence Interval: 611
Upper Confidence Interval: 695


In [ ]:
product_counts = df_merged["product_id"].value_counts()
relevant_products = product_counts[product_counts > low_conf].index
relevant_products

Index([24852, 13176, 21137, 21903, 47209, 47766, 47626, 16797, 26209, 27845,
       ...
       20197, 23495, 42221, 32420, 17070, 24920, 18102,  4933, 30278,  5386],
      dtype='int64', name='product_id', length=7248)

In [ ]:
df_merged = df_merged[df_merged["product_id"].isin(relevant_products)]
df_merged.shape


(28214831, 5)

In [ ]:
df_merged["user_id"].value_counts()
df_merged["user_id"].value_counts().mean()
low_conf, up_conf = sms.DescrStatsW(df_merged["user_id"].value_counts()).tconfint_mean()
print(f"Lower Confidence Interval: {low_conf:.0f}")
print(f"Upper Confidence Interval: {up_conf:.0f}")

Lower Confidence Interval: 136
Upper Confidence Interval: 138


In [ ]:
product_counts = df_merged["user_id"].value_counts()
relevant_users = product_counts[product_counts > low_conf].index
relevant_users

Index([201268, 129928, 186704, 182401, 137629, 176478, 164055,  79106,  60694,
        13701,
       ...
        94901,  94914,  23417, 176623,  15631, 187840, 173117, 173129,   7907,
        78709],
      dtype='int64', name='user_id', length=60532)

In [ ]:
df_merged = df_merged[df_merged["user_id"].isin(relevant_users)]
df_merged.shape

(20528009, 5)

In [ ]:
#df_filtered = df_merged[df_merged["product_id"].isin(relevant_products)]
#df_filtered = df_filtered[df_filtered["user_id"].isin(relevant_users)]

In [ ]:
basket = df_merged.groupby(["user_id","product_id"])["order_id"].count().unstack().notnull()
basket

product_id,1,10,23,25,28,34,37,45,49,54,...,49615,49621,49628,49640,49644,49652,49655,49667,49680,49683
user_id,,,,,,,,,,,,,,,,,,,,,
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
17,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206201,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
206202,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
206206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### Apriori Algorithm


In [ ]:
!pip install mlxtend


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00


In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(basket,min_support=0.1,use_colnames=True,verbose=1)
frequent_itemsets.sort_values("support", ascending=False)

Processing 580 combinations | Sampling itemset size 5


,support,itemsets
53,0.545893,(24852)
40,0.531834,(21137)
22,0.499851,(13176)
43,0.464862,(21903)
57,0.417449,(26209)
...,...,...
1079,0.100211,"(21137, 45007, 24964, 21903)"
686,0.100211,"(26209, 5876, 21903)"
366,0.100079,"(21616, 40706)"
469,0.100063,"(24852, 39877)"


In [ ]:
num_itemsets = len(frequent_itemsets)
rules = association_rules(frequent_itemsets, num_itemsets, metric="support", min_threshold=0.01)
rules.sort_values(by="lift", ascending=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3665,"(13176, 24964)","(47209, 22935)",0.216993,0.209691,0.102673,0.473163,2.256482,1.0,0.057172,1.500103,0.711146,0.316882,0.333379,0.481402
3668,"(47209, 22935)","(13176, 24964)",0.209691,0.216993,0.102673,0.489640,2.256482,1.0,0.057172,1.534225,0.704575,0.316882,0.348205,0.481402
3666,"(13176, 22935)","(47209, 24964)",0.215737,0.211657,0.102673,0.475917,2.248533,1.0,0.057011,1.504234,0.708010,0.316188,0.335210,0.480505
3667,"(47209, 24964)","(13176, 22935)",0.211657,0.215737,0.102673,0.485092,2.248533,1.0,0.057011,1.523113,0.704345,0.316188,0.343450,0.480505
3103,"(46667, 22935)",(24964),0.136275,0.351302,0.104028,0.763365,2.172961,1.0,0.056154,2.741348,0.624966,0.271224,0.635216,0.529743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,(24852),"(13176, 21137)",0.545893,0.322953,0.145477,0.266493,0.825176,1.0,-0.030821,0.923027,-0.318127,0.201110,-0.083392,0.358475
218,(13176),(24852),0.499851,0.545893,0.222031,0.444195,0.813703,1.0,-0.050834,0.817025,-0.314018,0.269549,-0.223952,0.425463
219,(24852),(13176),0.545893,0.499851,0.222031,0.406730,0.813703,1.0,-0.050834,0.843038,-0.335184,0.269549,-0.186186,0.425463
1698,"(13176, 47209)",(24852),0.273492,0.545893,0.116897,0.427424,0.782981,1.0,-0.032400,0.793094,-0.276155,0.166404,-0.260884,0.320781


In [ ]:
products=pd.read_csv('products.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_product_name(product_id, products_df):
    try:
        product_name = products_df.loc[products_df["product_id"] == product_id, "product_name"].iloc[0]
        return product_name
    except IndexError:
        return None

# Association Rule Learning (ARL) Recommender Function
def arl_recommender(rules_df, id, rec=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, k in enumerate(sorted_rules["antecedents"]):
        if id in list(k):
            for consequent in list(sorted_rules.iloc[i]["consequents"]):
                if consequent not in recommendation_list:
                    recommendation_list.append(consequent)
    return recommendation_list[:rec]

# Function to fetch product names for recommendations
def names_of_products(rules_df, bought, recommend=5):
    rec = arl_recommender(rules_df, bought, recommend)
    name_of_rec = {}

    # Fetch the name of the bought product
    bought_name = get_product_name(bought, products)
    if bought_name is None:
        return f"Product ID {bought} not found in the dataset."

    # Fetch the names of recommended products
    for i in rec:
        recommended_name = get_product_name(i, products)
        name_of_rec[i] = recommended_name if recommended_name else "Unknown Product"

    recommend_df = pd.DataFrame(name_of_rec.items(), columns=["product_id", "product_name"])
    print(f"Bought: {bought_name}\n")
    return recommend_df


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
product_id = int(input("Enter the product ID: "))

if get_product_name(product_id, products):
    recommendations = names_of_products(rules, product_id, 5)
    print(recommendations)
else:
    print(f"Product ID {product_id} not found in the dataset.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter the product ID: 47209
Bought: Organic Hass Avocado

   product_id            product_name
0       13176  Bag of Organic Bananas
1       24964          Organic Garlic
2       22935    Organic Yellow Onion
3       21903    Organic Baby Spinach
4       21137    Organic Strawberries


#### H-Mine Algorithm

In [ ]:
from mlxtend.frequent_patterns import hmine
frequent_itemsets = hmine(basket,min_support=0.05,use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
11508,0.545893,(24852)
6871,0.531834,(21137)
3534,0.499851,(13176)
9244,0.464862,(21903)
12523,0.417449,(26209)
...,...,...
857,0.050007,"(4920, 49683, 47766)"
10065,0.050007,"(26209, 47209, 47626, 21903, 47766)"
5048,0.050007,"(13176, 26209, 47626, 46667)"
9542,0.050007,"(24489, 26209, 21903)"


In [ ]:
num_itemsets = len(frequent_itemsets)
rules = association_rules(frequent_itemsets, num_itemsets, metric="support", min_threshold=0.01)
rules.sort_values(by="lift", ascending=False)
def get_product_name(product_id, products_df):
    try:
        product_name = products_df.loc[products_df["product_id"] == product_id, "product_name"].iloc[0]
        return product_name
    except IndexError:
        return None

# Association Rule Learning (ARL) Recommender Function
def arl_recommender(rules_df, id, rec=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []

    # Check both antecedents and consequents
    for i, (antecedent, consequent) in enumerate(zip(sorted_rules["antecedents"], sorted_rules["consequents"])):
        if id in antecedent:
            for product in consequent:
                if product not in recommendation_list:
                    recommendation_list.append(product)
        elif id in consequent:
            for product in antecedent:
                if product not in recommendation_list:
                    recommendation_list.append(product)

    return recommendation_list[:rec]

# Generate product names for recommendations
def names_of_products(rules_df, bought, recommend=5):
    rec = arl_recommender(rules_df, bought, recommend)
    name_of_rec = {}

    # Fetch the name of the bought product
    bought_name = get_product_name(bought, products)
    if bought_name is None:
        print(f"Product ID {bought} not found in the dataset.")
        return pd.DataFrame(columns=["product_id", "product_name"])  # Return an empty DataFrame

    # Fetch the names of recommended products
    for i in rec:
        recommended_name = get_product_name(i, products)
        if recommended_name:
            name_of_rec[i] = recommended_name
        else:
            print(f"Warning: Product ID {i} not found in products dataset.")
            name_of_rec[i] = "Unknown Product"

    recommend_df = pd.DataFrame(name_of_rec.items(), columns=["product_id", "product_name"])
    print(f"Bought: {bought_name}\nRecommendations:\n{recommend_df}\n")
    return recommend_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
product_id = int(input("Enter the product ID: "))

if get_product_name(product_id, products):
    recommendations = names_of_products(rules, product_id, 5)
else:
    print(f"Product ID {product_id} not found in the dataset.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter the product ID: 47209
Bought: Organic Hass Avocado
Recommendations:
   product_id            product_name
0       21137    Organic Strawberries
1        5876           Organic Lemon
2       22935    Organic Yellow Onion
3       24964          Organic Garlic
4       13176  Bag of Organic Bananas



#### FP Growth

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets = fpgrowth(basket,min_support=0.1,use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.545893,(24852)
16,0.531834,(21137)
1,0.499851,(13176)
69,0.464862,(21903)
17,0.417449,(26209)
...,...,...
523,0.100211,"(21137, 45007, 24964, 21903)"
790,0.100211,"(26209, 5876, 21903)"
1037,0.100079,"(21616, 40706)"
256,0.100063,"(24852, 39877)"


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
num_itemsets = len(frequent_itemsets)
rules = association_rules(frequent_itemsets, num_itemsets, metric="support", min_threshold=0.01)
rules.sort_values(by="lift", ascending=False)
def get_product_name(product_id, products_df):
    try:
        product_name = products_df.loc[products_df["product_id"] == product_id, "product_name"].iloc[0]
        return product_name
    except IndexError:
        return None

# Association Rule Learning (ARL) Recommender Function
def arl_recommender(rules_df, id, rec=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []

    # Check both antecedents and consequents
    for i, (antecedent, consequent) in enumerate(zip(sorted_rules["antecedents"], sorted_rules["consequents"])):
        if id in antecedent:
            for product in consequent:
                if product not in recommendation_list:
                    recommendation_list.append(product)
        elif id in consequent:
            for product in antecedent:
                if product not in recommendation_list:
                    recommendation_list.append(product)

    return recommendation_list[:rec]

# Generate product names for recommendations
def names_of_products(rules_df, bought, recommend=5):
    rec = arl_recommender(rules_df, bought, recommend)
    name_of_rec = {}

    # Fetch the name of the bought product
    bought_name = get_product_name(bought, products)
    if bought_name is None:
        print(f"Product ID {bought} not found in the dataset.")
        return pd.DataFrame(columns=["product_id", "product_name"])  # Return an empty DataFrame

    # Fetch the names of recommended products
    for i in rec:
        recommended_name = get_product_name(i, products)
        if recommended_name:
            name_of_rec[i] = recommended_name
        else:
            print(f"Warning: Product ID {i} not found in products dataset.")
            name_of_rec[i] = "Unknown Product"

    recommend_df = pd.DataFrame(name_of_rec.items(), columns=["product_id", "product_name"])
    print(f"Bought: {bought_name}\nRecommendations:\n{recommend_df}\n")
    return recommend_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
product_id = int(input("Enter the product ID: "))

if get_product_name(product_id, products):
    recommendations = names_of_products(rules, product_id, 5)
else:
    print(f"Product ID {product_id} not found in the dataset.")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter the product ID: 47209
Bought: Organic Hass Avocado
Recommendations:
   product_id            product_name
0       13176  Bag of Organic Bananas
1       24964          Organic Garlic
2       22935    Organic Yellow Onion
3       21903    Organic Baby Spinach
4       21137    Organic Strawberries



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
